In [43]:
import pandas as pd
from utils import LoadAndTransform
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [44]:
train_data = LoadAndTransform("train_sequences.csv", "train_labels.csv")
valid_data = LoadAndTransform("validation_sequences.csv", "validation_labels.csv")

In [52]:
train_data

,sequence,ID,resname,resid,x_1,y_1,z_1
0,GGGUGCUCAGUACGAGAGGAACCGCACCC,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,GGGUGCUCAGUACGAGAGGAACCGCACCC,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,GGGUGCUCAGUACGAGAGGAACCGCACCC,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,GGGUGCUCAGUACGAGAGGAACCGCACCC,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,GGGUGCUCAGUACGAGAGGAACCGCACCC,1SCL_A_5,G,5,1.827,-20.136000,11.793
...,...,...,...,...,...,...,...
137090,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,8Z1F_T_82,U,82,NaN,NaN,NaN
137091,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,8Z1F_T_83,C,83,NaN,NaN,NaN
137092,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,8Z1F_T_84,A,84,NaN,NaN,NaN
137093,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,8Z1F_T_85,U,85,NaN,NaN,NaN


In [53]:
X_train

,p_A,p_C,p_G,p_U,A,C,G,U
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
137070,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
137071,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
137072,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
137073,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Transformacoes para previsao

In [45]:
def CurrentAndPreviousOneHot(dataframe):
    dataframe = dataframe[dataframe["resname"].isin(list("ACGU"))]
    dataframe.loc[:, "previous_resname"] = dataframe["resname"].shift(1)
    encoder = OneHotEncoder()
    dataframe.loc[:,list("ACGU")] = encoder.fit_transform(dataframe[["resname"]]).toarray()
    dataframe.loc[:,["p_" + x for x in "ACGU"]] = encoder.fit_transform(dataframe[["resname"]]).toarray()
    dataframe = dataframe.dropna()

    return dataframe[
        ["p_" + x for x in "ACGU"] + 
        list("ACGU") + 
        [x + "_1" for x in "xyz"]
    ]

## First Keras model

In [46]:
train_data_onehot = CurrentAndPreviousOneHot(train_data)
valid_data_onehot = CurrentAndPreviousOneHot(valid_data)

/tmp/ipykernel_14770/1747014584.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.loc[:, "previous_resname"] = dataframe["resname"].shift(1)


In [47]:
X_train = train_data_onehot[["p_" + x for x in "ACGU"] + list("ACGU")]
X_valid = valid_data_onehot[["p_" + x for x in "ACGU"] + list("ACGU")]

y_train = train_data_onehot[[x + "_1" for x in "xyz"]]
y_valid = valid_data_onehot[[x + "_1" for x in "xyz"]]

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

y_train = scaler.fit_transform(y_train)
y_valid = scaler.transform(y_valid)